Dr. Jesús Martí Gavilá. Departamento de Ingeniería Cartográfica, Geodesia y fotogrametría
<font color="blue" ><h1 align=center>PRÁCTICA 7</h1> <p> <h1 align=center> CURSO BÁSICO PYTHON: FICHERO GPX</h1></font>

<a href="http://www.upv.es/titulaciones/GTI/indexc.html"><img src = "http://www.upv.es/imagenes/marcaUPVN1.png" width =200 ></a>

<h2 align=center>Grado Tecnologías Interactivas | Tecnologías de la Información Geográfica</h2>



# TAREA
<div class="alert alert-block alert-info" style="margin-top: 20px">

<b>Realizar un análisis de rendimiento a partir de un fichero GNSS con waypoints en formato GPX.</b>
<p>
    
<h3>El resultado a obtener sera:</h3>  <p> 
   <li>
       <a>Un Dataframe en formato HTML con los siguientes campos:<p><p>
'Punto','Latitud','Longitud', 'Cota (m)', 'Hora', 'Distancia (m)','Dist_O (m)', 'Pendiente (%)', 'Velocidad (km/h)','Orientación (º)','Tiempo'</a><p>
   </li>
        
   <li>
       <a>Gráfico con los valores de distancia en el eje X y valores de pendiente y velocidad en la Y</a>
   </li>
</div>

# Estructura GPX e importación de librerías

In [2]:

"""
Creado Viernes 14 Oct 18:22:39 2022, @autores: 
FORMATO EJEMPLO GPX
<wpt lat="38.968484" lon="-0.148688">
	<ele>-6.847729</ele>
	<time>2020-10-06T15:42:19Z</time>
	<name>001</name>
	<sym>Flag, Blue</sym>
</wpt>
"""
import pandas as pd
import xml.etree.cElementTree as et
import os
import math as m
from datetime import datetime
import matplotlib.pyplot as plt

# Fichero de entrada y Dataframe de salida

In [3]:
# Introducción de datos. Solicitar al usuario el nombre del fichero sin extensión.
path = os.getcwd();

nombregpx = input("Introduce el nombre del archivo GPX (sin extensión): ");

pathgpx = path + "/" + nombregpx + ".gpx";

# Este nombre también se utilizará para crear el fichero html
fichero_html = path + "/" + nombregpx + ".html";

Introduce el nombre del archivo GPX (sin extensión): Waypoints


In [5]:
# Accedemos al XML y creamos un Dataframe con los campos arriba indicados

docxml = et.parse(pathgpx)
raiz = docxml.getroot()
puntos = len(raiz)-1
columnas = ['Punto','Latitud','Longitud', 'Cota (m)', 'Hora', 'Distancia (m)',
                           'Dist_O (m)', 'Pendiente (%)', 'Velocidad (km/h)','Orientación (º)','Tiempo']
df = pd.DataFrame(columns=columnas)

# Funciones de cálculo

In [6]:
def calculard(angulo_a, angulo_b, landa):
    return m.acos((m.sin(angulo_a)*m.sin(angulo_b))+(m.cos(angulo_a)*m.cos(angulo_b)*m.cos(landa)));
    
# Cálculo de la distancia parcial y al origen
def distancia_parcial(d):
    return d*111.11;

In [20]:
# Cálculo del acimut. Verificar la posición del punto B
def acimut(angulo_a, angulo_b, d):
    return m.acos((m.sin(angulo_b)-m.sin(angulo_a)*m.cos(d))/(m.cos(angulo_a)*m.sin(d)));

In [8]:
# Cálculo de la pendiente. En %
def pendiente(h, dr):
    return (h/dr)*100;

In [9]:
# Cálculo de la velocidad
def velocidad(d, t):
    return d/t;

# Obtención de los datos de entrada y llamada a funciones

In [22]:
# Datos del punto A
wptA = raiz[1];
puntoA = {'lat':m.degrees(float(wptA.get("lat"))), 'lon':m.degrees(float(wptA.get("lon"))), 'ele':float(wptA[0].text), 'time':wptA[1].text, 'name':wptA[2].text, 'sym':wptA[3].text};
tiempo_A = datetime.strptime(puntoA['time'], '%Y-%m-%dT%H:%M:%SZ');
# tiempo_A_segundos = tiempo_A.total_seconds();
# print(tiempo_A_segundos);

In [23]:
# Datos del punto B
wptB = raiz[2];
puntoB = {'lat':m.degrees(float(wptB.get("lat"))), 'lon':m.degrees(float(wptB.get("lon"))), 'ele':float(wptB[0].text), 'time':wptB[1].text, 'name':wptB[2].text, 'sym':wptB[3].text};
tiempo_B = datetime.strptime(puntoB['time'], '%Y-%m-%dT%H:%M:%SZ');
diferencia=(tiempo_B-tiempo_A).total_seconds()
print (diferencia)

156.0


In [30]:
# Llamar a las diferentes funciones y añadir los datos resultantes al Dataframe
contador = 0;
lista = []
for punto in raiz:
    if contador==0:
        contador+=1;
    else:
        wptA = punto;
        lista.append({'lat':m.degrees(float(wptA.get("lat"))), 'lon':m.degrees(float(wptA.get("lon"))), 'ele':float(wptA[0].text), 'time':wptA[1].text, 'name':wptA[2].text, 'sym':wptA[3].text});
        contador+=1;

def calculos_entre_dos_puntos(punto1, punto2):
    d = calculard(punto1['lat'], punto2['lat'], (punto2['lon']+punto1['lon']));
    a = acimut(punto1['lat'], punto2['lat'], d);
    dp = distancia_parcial(d);
    v = velocidad(dp, diferencia);
    return {'d': d, 'a': a, 'dp': dp, 'v': v};

contador = 0;
lista = []
for punto in raiz:
    if contador==0:
        contador+=1;
    else:
        wptA = punto;
        lista.append({'lat':m.degrees(float(wptA.get("lat"))), 'lon':m.degrees(float(wptA.get("lon"))), 'ele':float(wptA[0].text), 'time':wptA[1].text, 'name':wptA[2].text, 'sym':wptA[3].text});
        contador+=1;

print(calculos_entre_dos_puntos(puntoA, puntoB));

{'d': 0.8850454522704363, 'a': 2.356771512558348, 'dp': 98.33740020176818, 'v': 0.6303679500113345}


# Impresión y exportación de los obtenidos

In [ ]:
# Imprimir los valores en formato HTML

In [ ]:
# Plotear el gráfico de rendimiento y guardar en JPG. Puedes ver ayuda en internet matplotlib.plot 
# para indicar leyendas, titulo, etiquetas del eje X

# Empieza con para plotear 2 ejes
fig, ax = plt.subplots()
plt.plot(df["Dist_O (m)"], df['Pendiente (%)'], marker = "o", label="Pendiente %")




# Mapa interactivo con Folium

In [ ]:
df.loc[:,['Latitud','Longitud']].to_csv('coordenadas.csv',header=True, index=False) #obtienes los valores de latitud y longitud y creas un fichero de coordenadas
lista_geo = pd.read_csv('coordenadas.csv').to_numpy().tolist()

import folium
from folium import plugins
m = folium.Map()
folium.plugins.AntPath(locations = lista_geo, popup = 'Ruta 20/11/2023', reverse="True", dash_array=[10, 20], color='red', pulse_color = 'blue').add_to(m)
m.fit_bounds(m.get_bounds())
m
